# Customer Churn with Explainable AI — Answer Key
This notebook trains a model, explains predictions, and shows subgroup metrics.

In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.inspection import permutation_importance

X = pd.read_csv("../data/customers.csv")
y = pd.read_csv("../data/labels.csv")["churned"]
cat_cols = ["region"]
num_cols = [c for c in X.columns if c not in ["customer_id"] + cat_cols]
pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols)
])
model = LogisticRegression(max_iter=500)
pipe = Pipeline([("pre", pre), ("clf", model)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
pipe.fit(X_train, y_train)
proba = pipe.predict_proba(X_test)[:,1]
print("ROC AUC:", roc_auc_score(y_test, proba).round(3))
print(classification_report(y_test, (proba>0.5).astype(int)))

# Explainability
try:
    import shap
    explainer = shap.Explainer(pipe, X_train)
    sv = explainer(X_test.sample(200, random_state=42))
    shap.plots.beeswarm(sv, show=False)
except Exception as e:
    print("SHAP unavailable, using permutation importances:", e)
    r = permutation_importance(pipe, X_test, y_test, n_repeats=5, random_state=42)
    importances = pd.Series(r.importances_mean, index=pipe.named_steps['pre'].get_feature_names_out())
    print(importances.sort_values(ascending=False).head(10))

# Simple subgroup metric
df_test = X_test.copy(); df_test["y"] = y_test.values; df_test["p"] = proba
for region in df_test["region"].unique():
    sub = df_test[df_test["region"]==region]
    auc = roc_auc_score(sub["y"], sub["p"])
    print(f"AUC for region={region}: {auc:.3f}")
